In [1]:
# Results w.r.t. different properties of the molecule

import pickle as pkl
import numpy as np
import matplotlib
#matplotlib.rcParams.update({'font.size': 40})
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd
import os
import time
from collections import OrderedDict
from summary_utils import read_mols_baseline, read_mols_neuralnet
%matplotlib inline

In [2]:
dataset = 'QM9'
num_conformers = [100]
nn_hyperparams = ["prior_T=1"]

In [3]:
ff_mols_orig = read_mols_baseline('/home/mansimov/projects/dl4chem-geometry/results/ff/{}/_test_/rep_100_iter_200/mols'.format(dataset), num_conformers)

done 0 out of 5000
done 500 out of 5000
done 1000 out of 5000
done 1500 out of 5000
done 2000 out of 5000
done 2500 out of 5000
done 3000 out of 5000
done 3500 out of 5000
done 4000 out of 5000
done 4500 out of 5000
time took 16.277498483657837


In [4]:
nn_ff_mols_newopt = read_mols_baseline('/home/mansimov/projects/dl4chem-geometry/results/nn_ff_rndcoord_nochiral_ignoresmooth/{}/_test_/rep_100_iter_200/mols'.format(dataset), num_conformers)

done 0 out of 4647
done 500 out of 4647
done 1000 out of 4647
done 1500 out of 4647
done 2000 out of 4647
done 2500 out of 4647
done 3000 out of 4647
done 3500 out of 4647
done 4000 out of 4647
done 4500 out of 4647
time took 11.190736055374146


In [5]:
def success_rate_permol(results, result_type):
    success = 0
    for k in results.keys():
        if result_type in results[k]:
            success += len(results[k][result_type])
    return ((success / (len(results.keys()) * 100)) * 100)

def total_count(results, result_type):
    count = 0
    for k in results.keys():
        if result_type in results[k]:
            count += 1
    return count

def parse_results(results, keys, result_type):
    mean, std, best = [], [], []
    for k in keys:
        mean.append(np.mean(results[k][result_type]))
        std.append(np.std(results[k][result_type]))
        best.append(np.min(results[k][result_type]))
    return np.median(mean), np.median(std), np.median(best)

overlap_keys = []
for k in nn_ff_mols_newopt.keys():
    if k in ff_mols_orig.keys():
        overlap_keys.append(k)

ff_mols_orig_mmff = parse_results(ff_mols_orig, overlap_keys, 'mmff')
nn_ff_mols_newopt_mmff = parse_results(nn_ff_mols_newopt, overlap_keys, 'mmff')

ff_mols_orig_uff = parse_results(ff_mols_orig, overlap_keys, 'uff')
nn_ff_mols_newopt_uff = parse_results(nn_ff_mols_newopt, overlap_keys, 'uff')


total_count_stats = ['total count (out of 5000)', total_count(ff_mols_orig, 'mmff'), total_count(nn_ff_mols_newopt, 'mmff'), total_count(ff_mols_orig, 'uff'), total_count(nn_ff_mols_newopt, 'uff')]
success_rate_stats = ['success percent (per mol)', success_rate_permol(ff_mols_orig, 'mmff'), success_rate_permol(nn_ff_mols_newopt, 'mmff'), success_rate_permol(ff_mols_orig, 'uff'), success_rate_permol(nn_ff_mols_newopt, 'uff')]
mean_stats = ['median of mean RMSD (mols present in all)', ff_mols_orig_mmff[0], nn_ff_mols_newopt_mmff[0], ff_mols_orig_uff[0], nn_ff_mols_newopt_uff[0]]
std_stats = ['median of std RMSD (mols present in all)', ff_mols_orig_mmff[1], nn_ff_mols_newopt_mmff[1], ff_mols_orig_uff[1], nn_ff_mols_newopt_uff[1]]
best_stats = ['median of best RMSD (mols present in all)', ff_mols_orig_mmff[2], nn_ff_mols_newopt_mmff[2], ff_mols_orig_uff[2], nn_ff_mols_newopt_uff[2]]

pd.DataFrame([total_count_stats, success_rate_stats, mean_stats, std_stats, best_stats], columns=['', 'MMFF original', 'NN + MMFF (random coord/no chiral)', 'UFF original', 'NN + UFF (random coord/no chiral)'])


,,MMFF original,NN + MMFF (random coord/no chiral),UFF original,NN + UFF (random coord/no chiral)
0,total count (out of 5000),4807.000000,3646.000000,4807.000000,3646.000000
1,success percent (per mol),98.787185,92.246846,98.787185,92.246846
2,median of mean RMSD (mols present in all),0.382852,0.242689,0.390513,0.269165
3,median of std RMSD (mols present in all),0.178316,0.065284,0.168663,0.053723
4,median of best RMSD (mols present in all),0.092186,0.099382,0.123120,0.132934
